In [1]:
# Example 20-1. bulkfood_v3.py: Quantity descriptors manage attributes in LineItem.
class Quantity:  # <1>
    def __init__(self, storage_name):
        self.storage_name = storage_name  # <2>
    def __set__(self, instance, value):  # <3>
        if value > 0:
            instance.__dict__[self.storage_name] = value  # <4>
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity('weight')  # <5>
    price = Quantity('price')  # <6>
    def __init__(self, description, weight, price):  # <7>
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price
    
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [9]:
# Example 20-2. bulkfood_v4.py: Each Quantity descriptor gets a unique storage_name
class Quantity:
    __counter = 0  # <1>
    def __init__(self):
        cls = self.__class__  # <2>
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)  # <3>
        cls.__counter += 1  # <4>
    def __get__(self, instance, owner):  # <5>
        return getattr(instance, self.storage_name)  # <6>
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)  # <7>
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity()  # <8>
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price
    
coconuts = LineItem('Brazilian coconut', 20, 17.95)
print("coconuts.weight, coconuts.price : ", coconuts.weight, ",", coconuts.price)
print("getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1') : ", 
     getattr(coconuts, '_Quantity#0'), ",", getattr(coconuts, '_Quantity#1'))
LineItem.weight

coconuts.weight, coconuts.price :  20 , 17.95
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1') :  20 , 17.95


AttributeError: 'NoneType' object has no attribute '_Quantity#0'

In [60]:
# Example 20-3. bulkfood_v4b.py (partial listing): When invoked through the managed
# class, __get__ returns a reference to the descriptor itself
class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self  # <1>
        else:
            return getattr(instance, self.storage_name)  # <2>
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity()  # <8>
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

print("LineItem.price : ", LineItem.price)           
br_nuts = LineItem('Brazil nuts', 10, 34.95)
print("br_nuts.price : ", br_nuts.price)
help(LineItem.weight)

LineItem.price :  <__main__.Quantity object at 0x7f054c56c208>
br_nuts.price :  34.95
Help on Quantity in module __main__ object:

class Quantity(builtins.object)
 |  Methods defined here:
 |  
 |  __get__(self, instance, owner)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __set__(self, instance, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [ ]:
# Example 20-6. model_v5.py: The refactored descriptor classes
import abc

class AutoStorage:  # <1>
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)  # <2>

class Validated(abc.ABC, AutoStorage):  # <3>
    def __set__(self, instance, value):
        value = self.validate(instance, value)  # <4>
        super().__set__(instance, value)  # <5>
    @abc.abstractmethod
    def validate(self, instance, value):  # <6>
        """return validated value or raise ValueError"""

class Quantity(Validated):  # <7>
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value  # <8>
    
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [16]:
# Example 20-8. - 9. descriptorkinds.py: Simple classes for studying descriptor overriding behaviors.

### auxiliary functions for display only ###
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

### essential classes for this example ###
class Overriding:  # <1>
    """a.k.a. data descriptor or enforced descriptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)  # <2>
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class OverridingNoGet:  # <3>
    """an overriding descriptor without ``__get__``"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:  # <4>
    """a.k.a. non-data or shadowable descriptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

class Managed:  # <5>
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    def spam(self):  # <6>
        print('-> Managed.spam({})'.format(display(self)))

obj = Managed()
print("obj.over : ", obj.over)
print("Managed.over : ", Managed.over, "\n")

obj.over = 7
print("vars(obj) : ", vars(obj))
print("obj.over : ", obj.over, "\n")

obj.__dict__['over'] = 8
print("vars(obj) : ", vars(obj))
print("obj.over : ", obj.over, "\n")

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
obj.over :  None
-> Overriding.__get__(<Overriding object>, None, <class Managed>)
Managed.over :  None 

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)
vars(obj) :  {}
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
obj.over :  None 

vars(obj) :  {'over': 8}
-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
obj.over :  None 



In [18]:
# Example 20-10. Overriding descriptor without __get__: obj.over_no_get is an instance of OverridingNoGet
# continued from 20-8.
obj = Managed()

print("obj.over_no_get : ", obj.over_no_get)
print("Managed.over_no_get : ", Managed.over_no_get, "\n")

obj.over_no_get = 7
print("obj.over_no_get : ", obj.over_no_get, "\n")

obj.__dict__['over_no_get'] = 9
print("obj.over_no_get : ", obj.over_no_get)

obj.over_no_get = 7
print("obj.over_no_get : ", obj.over_no_get)

obj.over_no_get :  <__main__.OverridingNoGet object at 0x7f054c5eb240>
Managed.over_no_get :  <__main__.OverridingNoGet object at 0x7f054c5eb240> 

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
obj.over_no_get :  <__main__.OverridingNoGet object at 0x7f054c5eb240> 

obj.over_no_get :  9
-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)
obj.over_no_get :  9


In [22]:
# Example 20-11. Behavior of a non-overriding descriptor: obj.non_over is an instanceof NonOverriding
# continued from 20-8.
obj = Managed()
print("obj.non_over : ", obj.non_over, "\n")

obj.non_over = 7
print("obj.non_over : ", obj.non_over)
print("Managed.non_over : ", Managed.non_over, "\n")

del obj.non_over
print("obj.non_over : ", obj.non_over)

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
obj.non_over :  None 

obj.non_over :  7
-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)
Managed.non_over :  None 

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)
obj.non_over :  None


In [24]:
# Example 20-12. Any descriptor can be overwritten on the class itself
# continued from 20-8.
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over  # The descriptors are really gone.

(1, 2, 3)

In [28]:
# Example 20-13. A method is a non-overriding descriptor
# continued from 20-8.
obj = Managed()
print("obj.spam : ", obj.spam)
print("Managed.spam : ", Managed.spam, "\n")

obj.spam = 7
print("obj.spam : ", obj.spam)

obj.spam :  <bound method Managed.spam of <__main__.Managed object at 0x7f054ce2d8d0>>
Managed.spam :  <function Managed.spam at 0x7f054ce478c8> 

obj.spam :  7


In [52]:
# Example 20-14. method_is_descriptor.py: a Text class, derived from UserString
import collections
class Text(collections.UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    def reverse(self):
        return self[::-1]

word = Text('forward')
print("word : %r" % word)
print("word.reverse() : %r" % word.reverse())
print("Text.reverse(Text('backward')) : %r" % Text.reverse(Text('backward')))
print("type(Text.reverse), type(word.reverse) : ", type(Text.reverse), type(word.reverse), "\n")

print("list( map( Text.reverse, ['repaid', (10, 20, 30), Text('stressed') ] )) : \n", 
     list( map( Text.reverse, ['repaid', (10, 20, 30), Text('stressed')])), "\n")

print("Text.reverse.__get__(None, Text) : ", Text.reverse.__get__(None, Text))
print("Text.reverse.__get__(word, Text) : ", Text.reverse.__get__(word, Text))
print("word.reverse : ", word.reverse, "\n")
print("word.reverse.__self__ : %r" % word.reverse.__self__)
print("word.reverse.__func__ is Text.reverse : ", word.reverse.__func__ is Text.reverse)

word : Text('forward')
word.reverse() : Text('drawrof')
Text.reverse(Text('backward')) : Text('drawkcab')
type(Text.reverse), type(word.reverse) :  <class 'function'> <class 'method'> 

list( map( Text.reverse, ['repaid', (10, 20, 30), Text('stressed') ] )) : 
 ['diaper', (30, 20, 10), Text('desserts')] 

Text.reverse.__get__(None, Text) :  <function Text.reverse at 0x7f0551683510>
Text.reverse.__get__(word, Text) :  <bound method Text.reverse of Text('forward')>
word.reverse :  <bound method Text.reverse of Text('forward')> 

word.reverse.__self__ : Text('forward')
word.reverse.__func__ is Text.reverse :  True


In [ ]:
print("obj.over_no_get : ", obj.over_no_get)
print(" : ", )

import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')